In [302]:
import pandas as pd
import requests
import pickle

In [2]:
url = 'https://travel.state.gov/content/travel/en/traveladvisories/traveladvisories.html'
html = requests.get(url).content
df_list = pd.read_html(html)
df = df_list[-1]

In [3]:
travelAdvisories = {}

for i in range(len(df[0])):
    if i == 0:
        continue
    
    country = df[0][i]
    level = df[1][i]
    travelAdvisories[country] = level

In [4]:
print(travelAdvisories)

{u'Canada': u'1: Exercise normal precautions', u'Turkmenistan': u'1: Exercise normal precautions', u'Lithuania': u'1: Exercise normal precautions', u'Cambodia': u'1: Exercise normal precautions', u'Switzerland': u'1: Exercise normal precautions', u'Ethiopia': u'2: Exercise increased caution', u'The Gambia': u'1: Exercise normal precautions', u'Aruba': u'1: Exercise normal precautions', u'Swaziland': u'1: Exercise normal precautions', u'Domincan Republic': u'2: Exercise increased caution', u'Bolivia': u'1: Exercise normal precautions', u'Cameroon': u'2: Exercise increased caution', u'Burkina Faso': u'3: Reconsider travel', u'Bahrain': u'1: Exercise normal precautions', u'Saudi Arabia': u'2: Exercise increased caution', u'Japan': u'1: Exercise normal precautions', u'Slovenia': u'1: Exercise normal precautions', u'Guatemala': u'3: Reconsider travel', u'Zimbabwe': u'2: Exercise increased caution', u'Bonaire, St. Eustatius, and Saba (BES)': u'1: Exercise normal precautions', u'Jordan': u'2:

In [303]:
with open('data/travelAdvisories.pickle', 'wb') as f:
    pickle.dump(travelAdvisories, f, protocol=2)

In [246]:
from netCDF4 import Dataset
import numpy as np
import sys

In [267]:
# These datasets are not on the GitHub repo as each are 140mb - 200mb.
# If it is desired to repickle the weather data, you can find this data here: http://berkeleyearth.org/data/
datasetAvg = Dataset('data/Complete_TAVG_LatLong1.nc')
datasetMin = Dataset('data/Complete_TMIN_LatLong1.nc')
datasetMax = Dataset('data/Complete_TMAX_LatLong1.nc')

In [268]:
print datasetAvg.file_format

NETCDF4


In [269]:
print datasetAvg.dimensions.keys()
print datasetAvg.dimensions['month_number']

[u'longitude', u'latitude', u'time', u'month_number']
<type 'netCDF4._netCDF4.Dimension'>: name = 'month_number', size = 12



In [270]:
print datasetAvg.variables.keys()

[u'longitude', u'latitude', u'time', u'land_mask', u'temperature', u'climatology']


In [271]:
print datasetAvg.variables['climatology'].shape
print datasetAvg.variables['climatology'].dimensions
print datasetAvg.variables['climatology'].ncattrs()
print datasetAvg.variables['temperature'].shape
print datasetAvg.variables['temperature'].dimensions
print datasetAvg.variables['temperature'].ncattrs()

print getattr(datasetAvg.variables['temperature'], 'standard_name')

(12, 180, 360)
(u'month_number', u'latitude', u'longitude')
[u'units', u'standard_name', u'long_name', u'coordinates', u'missing_value', u'valid_min', u'valid_max']
(3218, 180, 360)
(u'time', u'latitude', u'longitude')
[u'units', u'standard_name', u'long_name', u'missing_value', u'valid_min', u'valid_max']
surface_temperature_anomaly


In [272]:
def convertCToF(c):
    return 9.0/5.0 * c + 32

In [273]:
temp = datasetAvg.variables['climatology'][11,90+46.2,180+6.14]
print(convertCToF(temp))

31.479205989837645


/Users/NoahKaplan/miniconda3/lib/python2.7/site-packages/ipykernel_launcher.py:1: UserWarning: WARNING: valid_min not used since it
cannot be safely cast to variable data type
  """Entry point for launching an IPython kernel.
/Users/NoahKaplan/miniconda3/lib/python2.7/site-packages/ipykernel_launcher.py:1: UserWarning: WARNING: valid_max not used since it
cannot be safely cast to variable data type
  """Entry point for launching an IPython kernel.


In [242]:
with open('data/destination_geocode.json') as f:
    geocode = json.load(f)

In [274]:
print(len(geocode.keys()))

392


In [280]:
def getTemps(ncData):
    temps = {}
    for region in geocode.keys():
        lat = geocode[region.lower()]['results'][0]['geometry']['location']['lat']
        lon = geocode[region.lower()]['results'][0]['geometry']['location']['lng']

        for month in range(12):
            temps[(region, month)] = ncData.variables['climatology'][month,90+lat,180+lon]
            
    return temps

In [281]:
avgTemps = getTemps(datasetAvg)
minTemps = getTemps(datasetMin)
maxTemps = getTemps(datasetMax)

/Users/NoahKaplan/miniconda3/lib/python2.7/site-packages/ipykernel_launcher.py:8: UserWarning: WARNING: valid_min not used since it
cannot be safely cast to variable data type
  
/Users/NoahKaplan/miniconda3/lib/python2.7/site-packages/ipykernel_launcher.py:8: UserWarning: WARNING: valid_max not used since it
cannot be safely cast to variable data type
  


In [313]:
def getPercentDataWithTemp(temps):
    count = 0.
    for key in temps.keys():
        if str(temps[key]) != "--":
            count = count + 1
    return count / len(temps)

In [314]:
print(getPercentDataWithTemp(avgTemps))
print(getPercentDataWithTemp(minTemps))
print(getPercentDataWithTemp(maxTemps))

0.885204081633
0.885204081633
0.885204081633


In [291]:
print(avgTemps)

{(u'armenia', 3): 6.2823863, (u'nepal', 7): 13.207391, (u'indonesia', 9): 26.204264, (u'sumatra', 3): 26.24337, (u'cayman islands', 11): masked, (u'tanzania', 8): 22.187649, (u'mali', 11): 21.599854, (u'uruguay', 7): 12.883909, (u'edinburgh', 4): 9.917376, (u'tahiti and french polynesia', 5): 25.082132, (u'andorra', 1): 2.1290054, (u'east timor', 9): 25.824106, (u'gambia', 11): 24.784885, (u'branson', 1): 3.746977, (u'lisbon', 2): 12.83597, (u'hobart', 11): 13.7599125, (u'houston', 11): 12.681468, (u'saint kitts and nevis', 10): masked, (u'belfast', 6): 13.63507, (u'curacao', 9): 28.635452, (u'usa', 9): 15.451993, (u'cook islands', 0): masked, (u'san francisco', 4): 14.624635, (u'stuttgart', 10): 3.494475, (u'pitcairn islands', 4): masked, (u'salzburg', 11): -3.9294746, (u'canberra', 10): 14.576279, (u'new zealand', 3): masked, (u'tonga', 1): masked, (u'minneapolis', 0): -11.87486, (u'cambodia', 2): 28.805119, (u'christchurch', 4): 7.6387625, (u'latvia', 2): -1.6138777, (u'turkmenistan

In [304]:
temps = {}

for region in avgTemps.keys():
    if str(avgTemps[region]) != "--":
        temps[region] = (minTemps[region], avgTemps[region], maxTemps[region])

In [309]:
print(temps)

{(u'armenia', 3): (0.48436937, 6.2823863, 11.921986), (u'nepal', 7): (7.4286633, 13.207391, 18.949942), (u'macau', 10): (14.214862, 20.038738, 25.824661), (u'sumatra', 3): (20.562952, 26.24337, 31.889412), (u'tanzania', 8): (16.386177, 22.187649, 27.905012), (u'mali', 11): (15.941875, 21.599854, 27.086874), (u'andalucia', 3): (8.997289, 14.622792, 20.28516), (u'edinburgh', 4): (4.204908, 9.917376, 15.596082), (u'croatia', 0): (-5.0472417, 0.5125486, 6.284817), (u'andorra', 1): (-3.3872879, 2.1290054, 7.8709283), (u'gambia', 11): (19.168547, 24.784885, 30.480951), (u'branson', 1): (-2.0024998, 3.746977, 9.554907), (u'mexico city', 9): (7.740504, 13.522783, 19.074018), (u'hobart', 11): (8.093201, 13.7599125, 19.4219), (u'houston', 11): (6.654963, 12.681468, 18.512802), (u'boston', 1): (-9.425438, -3.6541088, 2.0721862), (u'curacao', 9): (22.944078, 28.635452, 34.33303), (u'usa', 9): (9.227174, 15.451993, 21.3813), (u'san francisco', 4): (8.987959, 14.624635, 20.485704), (u'stuttgart', 10

In [306]:
with open('data/temperatures.pickle', 'wb') as f:
    pickle.dump(temps, f, protocol=2)

In [307]:
# Good resource for contemporary average temperatures ONLY in the the US
url = 'https://www.melissadata.com/lookups/ZipWeather.asp?ZipCode=14850'
html = requests.get(url).content
df_list = pd.read_html(html)
df = df_list[1]

def getWeatherFromMelissaDFList(df_list) :
    ret = {}
    
    for i in range(12):
        highTemp = df_list[4][1][i+4]
        avgTemp = df_list[4][2][i+4]
        lowTemp = df_list[4][3][i+4]
        precip = df_list[4][6][i+4]
        
        ret[i] = (highTemp, avgTemp, lowTemp, precip)
        
    return ret

In [308]:
print(getWeatherFromMelissaDFList(df_list))

{0: (u'31.2', u'22.6', u'13.9', u'2.1'), 1: (u'33.2', u'23.5', u'13.7', u'2.1'), 2: (u'42.3', u'32.4', u'22.4', u'2.6'), 3: (u'54.5', u'43.8', u'33.1', u'3.3'), 4: (u'67.3', u'55.4', u'43.4', u'3.2'), 5: (u'75.7', u'64.4', u'53.1', u'3.9'), 6: (u'80.1', u'68.7', u'57.2', u'3.5'), 7: (u'78.7', u'67.3', u'55.9', u'3.4'), 8: (u'70.9', u'59.4', u'47.9', u'3.8'), 9: (u'59.4', u'48.4', u'37.3', u'3.2'), 10: (u'47.2', u'38.9', u'30.6', u'3.1'), 11: (u'36.1', u'28.2', u'20.3', u'2.5')}
